# **การดึงข้อมูลตลาดหุ้น ที่มีหุ้นหลายๆตัวพร้อมกัน และจัดทำ multi level index**

เริ่มจากการ install yfinance

In [ ]:
#คำสั่งสำหรับการ install yfinance
! pip install yfinance

จากนั้น เข้าไปดู ticker ในเว็บไซต์ yahoofinance.com
แล้วทำการ download ข้อมูลมาเก็บไว้เป็นไฟล์ .csv

In [ ]:
import yfinance as yf
import datetime as dt # ไลบรารีที่เราจะใช้สำหรับงานที่เกี่ยวข้องกับ วันเวลา
import pandas as pd # ไลบรารีที่เราจะใช้ทำงานเกี่ยวกับตาราง (Dataframe)
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina' #กำหนดรูปกราฟเป็นความละเอียดสูง

In [ ]:
print(f"yahoo finance version = {yf.__version__}")

# ดึงข้อมูลหุ้นหลายๆตัวพร้อมกัน
**การเรียกข้อมูลแบบ multilevel column names**

In [ ]:
stocks=['EA.BK', 'AOT.BK', 'SCB.BK'] #ดึงหุ้นไทยมากกว่า 1 ตัว
#start = dt.datetime(2022, 5, 1)
#end = dt.datetime(2023, 4, 30)
df2 = yf.download(stocks, start="2022-05-01", end="2023-04-30")
df2.head()

In [ ]:
df2.info()

In [ ]:
df2.index

In [ ]:
#แก้ไข index เพื่อเปลี่ยน dtype ของ index เป็น datetime64[ns]
df2.index = df2.index.tz_localize(None)

In [ ]:
df2.index

In [ ]:
temp_data = pd.DataFrame(df2) #backup data df2

In [ ]:
temp_data.head(10)

In [ ]:
df2.info()

ดูข้อมูลค่า High ของทุกหุ้น

In [ ]:
df2['High'].head()

ดูข้อมูลค่า High ของหุ้น 'EA'

In [ ]:
df2['High']['EA.BK'].head()

ดูข้อมูลตั้งแต่วันที่ต้องการ ทุกคอลัมน์

In [ ]:
df2.loc['2022-08-13':]

In [ ]:
df2.loc['2022-08-15':]['Open']

In [ ]:
df2.loc['2022-08-15':, ('Open', 'AOT.BK')]

In [ ]:
df2.loc['2022-08-15':, ('Close', ['AOT.BK', 'SCB.BK'])]

In [ ]:
df2.loc['2022-08-15':'2022-08-22', ('Close', ['AOT.BK', 'SCB.BK'])]

In [ ]:
df2.loc['2022-08-15':'2022-08-22', (['Open', 'Close'], ['AOT.BK', 'SCB.BK'])]

In [ ]:
#บันทึกข้อมูลลงไฟล์
df2.to_csv('mlevel_stock.csv')

In [ ]:
df2.to_excel('mlevel_stock.xlsx')

# stack DataFrame

In [ ]:
df2.stack() #จัดข้อมูลเรียงแบบ stack เพื่อให้อ่านง่ายขึ้น กรณีข้อมูลเป็น multilevel

In [ ]:
dstack = df2.stack().reset_index()
dstack

In [ ]:
dstack.sort_values(['Ticker','Date'])

In [ ]:
dstack[dstack.Ticker=='SCB.BK']

In [ ]:
sns.lineplot(x='Date', y='Close', hue='Ticker',
             style='Ticker',data=dstack); #ถ้าไม่ใส่เครื่องหมาย ; ด้านหลังจะเกิดอะไรขึ้น???

In [ ]:
sns.lineplot(x='Date', y='Close', hue='Ticker',
             style='Ticker',data=dstack)

Relationship plot แสดงกราฟ

In [ ]:
sns.relplot(x='Date', y='Close', col='Ticker',
           hue='Ticker', kind='line',
           height=4, #ความสูงของกราฟ
           facet_kws={'sharey': False},
           #ปรับ scale ของ y-axis ตามช่วงข้อมูล โดยไม่ share ค่าแกน y ร่วมกัน
           data=dstack);

plot เฉพาะบางหุ้นที่ต้องการ

In [ ]:
sns.lineplot(x='Date', y='Close',
           hue='Ticker',
           data=dstack[dstack.Ticker.str.match('EA.BK|AOT.BK')]);

In [ ]:
dstack.to_csv('stack_port.csv', index = False)
dstack.to_excel('stack_port.xlsx', index = False)

ดึงข้อมูล stooq

In [ ]:
import pandas_datareader.data as web

df = web.get_data_stooq(['^DJI','^SET'])

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df.columns.get_level_values(0)

In [ ]:
df.columns.get_level_values(1)

In [ ]:
cols = df.columns.get_level_values(1)+ '_' + df.columns.get_level_values(0)
cols

In [ ]:
cols = cols.str.replace('^','')
cols

In [ ]:
df.columns

In [ ]:
df.columns = cols #flatten multi-level columns

In [ ]:
df.columns #เหลือ index level เดียว

In [ ]:
df.tail()

เปลี่ยน multiindex ของหุ้นไทย 3 ตัวให้เป็น index เดียว

In [ ]:
df2.columns

In [ ]:
cols_2 = df2.columns.get_level_values(1)+ '_' + df2.columns.get_level_values(0)
print(cols_2)

In [ ]:
df2.columns = cols_2

In [ ]:
df2.head()

In [ ]:
df2.columns

ปรับชื่อ คอลัมน์อีกรูปแบบหนึ่ง โดยใช้ ข้อมูล df2 ตัวตั้งต้น

In [ ]:
#stocks=['EA.BK', 'AOT.BK', 'SCB.BK'] #ดึงหุ้นไทยมากกว่า 1 ตัว
#df2 = yf.download(stocks, start="2022-05-01", end="2023-04-30")
df2 = pd.DataFrame(temp_data)

In [ ]:
df2.columns

In [ ]:
cols_3 = df2.columns.get_level_values(0)+ '_' + df2.columns.get_level_values(1)
print(cols_3)

In [ ]:
temp_data.head()

In [ ]:
df2.head()

In [ ]:
df2.columns = cols_3

In [ ]:
df2.head()

In [ ]:
df2.columns

In [ ]:
temp_data.columns